# # 모듈 및 함수
---

In [45]:
import pandas as pd
import numpy as np
import os
import open3d as o3d
import pickle as pkl
import re
import json
import shutil
import glob
from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split


def xyxy2xywhn(x, w=1920, h=1200, clip=False, eps=0.0):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    if clip:
        clip_boxes(x, (h - eps, w - eps))  # warning: inplace clip
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = ((x[:, 0] + x[:, 2]) / 2) / w  # x center
    y[:, 1] = ((x[:, 1] + x[:, 3]) / 2) / h  # y center
    y[:, 2] = (x[:, 2] - x[:, 0]) / w  # width
    y[:, 3] = (x[:, 3] - x[:, 1]) / h  # height
    y = list(y.reshape(-1))
    return y

# rotation matrix
def roty(t, Rx=90/180*np.pi):
    ''' Rotation about the y-axis. '''
    c = np.cos(t)
    s = np.sin(t)
    
    X = np.array([[1, 0, 0],
                    [0, np.cos(Rx), -np.sin(Rx)],
                    [0, np.sin(Rx), np.cos(Rx)]])

    Z = np.array([[c, -s, 0],
                    [s, c, 0],
                    [0, 0, 1]])
    
    return np.matmul(Z, X)

def xyz2xyxy(x, y, z, l, w, h, rot_y, extrinsic, intrinsic):
    R = roty(rot_y)
   
    x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
    y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
    z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];
    
    corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
    corners_3d[0, :] = corners_3d[0, :] + x  # x
    corners_3d[1, :] = corners_3d[1, :] + y  # y
    corners_3d[2, :] = corners_3d[2, :] + z  # z
    corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])
    
    point2d = np.matmul(intrinsic, np.matmul(extrinsic, corners_3d))
    pointx = np.around(point2d/point2d[2])[0]
    pointy = np.around(point2d/point2d[2])[1]

    return min(pointx), min(pointy), max(pointx), max(pointy)

# # 데이터 프레임 만들기
---

In [50]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final'


# Z축 이동을 위해서 calib와 매칭하여 이동범위 지정
calib_ls = []
# scenes = []
calibs = sorted(glob.glob(f'{src}/*/calib/camera/camera_0.json'))
for calib in calibs:
    scene = re.findall('[a-zA-Z0-9_]+', calib)[-5]
    with open(calib, 'r') as f:
        calib = json.load(f)
    if calib['extrinsic'] not in calib_ls:
        calib_ls.append(calib['extrinsic'])
        # scenes.append(scene)    
calib_typ = {'typ1': {'calib': calib_ls[0], 'mov_zpoint': 14},
             'typ2': {'calib': calib_ls[1], 'mov_zpoint': 13},
             'typ3': {'calib': calib_ls[2], 'mov_zpoint': 0},
             'typ4': {'calib': calib_ls[3], 'mov_zpoint': -20}}


# 라벨데이터로 데이터프레임 생성
dp_ls = []
labels = sorted(glob.glob(f'{src}/*/label/*.json'))
for j, label in enumerate(tqdm(labels)):
    scene = re.findall('\w+', label)[-4]
    frame = re.findall('\w+', label)[-2]

    # calib값 조정
    with open(f'{src}/{scene}/calib/camera/camera_0.json', 'r') as f:
        calib = json.load(f)

    extrinsic = np.asarray(calib['extrinsic']).reshape(4, 4)
    intrinsic = np.zeros([3, 4])
    intrinsic[:3, :3] = np.asarray(calib['intrinsic']).reshape(3, 3)
    
    for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
        if calib['extrinsic'] == calib_typ[typ]['calib']:
            # extrinsic = np.asarray(calib['extrinsic']).reshape(4, 4)
            extrinsic[:3, 3] -= extrinsic[:3, 2] * calib_typ[typ]['mov_zpoint']   

    # 컬럼 구성
    globals()[f'dp{j}'] = pd.DataFrame()
    with open(label, 'r') as f:
        json_data = json.load(f)

    for i in range(len(json_data)):
        try:
            id_ = json_data[i]['obj_id']
            class_ = json_data[i]['obj_type']
            psr = json_data[i]['psr']
            point_x, point_y, point_z = psr['position']['x'], psr['position']['y'], psr['position']['z']
            # z값 범위를 줄이기 위해 조정
            for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
                if calib['extrinsic'] == calib_typ[typ]['calib']:
                    mov_point_z = calib_typ[typ]['mov_zpoint']
                    point_z += mov_point_z
            l, w, h = psr['scale']['x'], psr['scale']['y'], psr['scale']['z']
            rot_y = psr['rotation']['z']
            min_x, min_y, max_x, max_y = xyz2xyxy(point_x, point_y, point_z, l, w, h, rot_y, extrinsic, intrinsic)

            data = [
                id_, class_, 
                min_x, min_y, max_x, max_y,
                point_x, point_y, point_z,
                l, w, h,
                rot_y,
                intrinsic.flatten().tolist(), extrinsic.flatten().tolist(), mov_point_z,
                scene, frame
                ]
            
            columns = [
                'id', 'class',
                'min_x', 'min_y', 'max_x', 'max_y',
                'point_x', 'point_y', 'point_z',
                'l', 'w', 'h',
                'rot_y',
                'intrinsic', 'extrinsic', 'mov_point_z',
                'scene', 'frame'
                ]

            frame_data = pd.DataFrame(data=[data], columns=columns)
            globals()[f'dp{j}'] = pd.concat([globals()[f'dp{j}'], frame_data], axis=0)
        except:
            continue
    dp_ls.append(globals()[f'dp{j}'])
    
dp = pd.concat(dp_ls).reset_index(drop=True)

dp.loc[dp['min_x'] < 0, 'min_x'] = 0
dp.loc[dp['min_x'] > 1920, 'min_x'] = 1920
dp.loc[dp['max_x'] < 0, 'max_x'] = 0
dp.loc[dp['max_x'] > 1920, 'max_x'] = 1920
dp.loc[dp['min_y'] < 0, 'min_y'] = 0
dp.loc[dp['min_y'] > 1200, 'min_y'] = 1200
dp.loc[dp['max_y'] < 0, 'max_y'] = 0
dp.loc[dp['max_y'] > 1200, 'max_y'] = 1200
dp['id'] = dp['id'].apply(pd.to_numeric, errors='coerce')
drop_index = dp.loc[(dp['class']==0) | (dp['class']=='Unknown') | (dp['id'].isnull()) | (dp['min_x']-dp['max_x']==0) | (dp['min_y']-dp['max_y']==0)].index
dp = dp.drop(drop_index).reset_index(drop=True)
dp['id'] = dp['id'].astype(int)
dp.to_csv(f'{dst}/data_info.csv')
# dp = pd.read_csv('/data/NIA50/50-2/data/nia50_final/data_info.csv', index_col=0, dtype={'frame':object})

100%|██████████| 59660/59660 [18:49<00:00, 52.81it/s] 


In [ ]:
# 라벨 체크
no_label = []
src = '/data/NIA50/50-2/data/nia50_final/raw'
labels = sorted(glob.glob(f'{src}/*/label/*.json'))
for j, label in enumerate(tqdm(labels)):
    scene = re.findall('\w+', label)[-4]
    frame = re.findall('\w+', label)[-2]

    with open(label, 'r') as f:
        json_data = json.load(f)

    if len(json_data)==0:
        no_label.append(scene)

from collections import Counter

for i in dict(Counter(no_label)).items():
    print(i)

In [ ]:
from collections import Counter

for i in dict(Counter(no_label)).items():
    print(i)

In [ ]:
# l, w, h 범위 확인
np.around(dp.groupby('class').quantile(0.99)[['l', 'w', 'h']].values, 2)

## # class 통합 1
---

In [81]:
dp = pd.read_csv('/data/NIA50/50-2/data/nia50_final/data_info.csv', index_col=0, dtype={'frame':object})
dp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 563424 entries, 0 to 563423
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           563424 non-null  int64  
 1   class        563424 non-null  object 
 2   min_x        563424 non-null  float64
 3   min_y        563424 non-null  float64
 4   max_x        563424 non-null  float64
 5   max_y        563424 non-null  float64
 6   point_x      563424 non-null  float64
 7   point_y      563424 non-null  float64
 8   point_z      563424 non-null  float64
 9   l            563424 non-null  float64
 10  w            563424 non-null  float64
 11  h            563424 non-null  float64
 12  rot_y        563424 non-null  float64
 13  intrinsic    563424 non-null  object 
 14  extrinsic    563424 non-null  object 
 15  mov_point_z  563424 non-null  int64  
 16  scene        563424 non-null  object 
 17  frame        563424 non-null  object 
dtypes: float64(11), int64(2)

In [82]:
# class 통합
dp.loc[(dp['class']=='Car') | (dp['class']=='Light_Car') | (dp['class']=='Small_Car'), 'class'] = 'Car'
dp.loc[(dp['class']=='SUV') | (dp['class']=='Van'), 'class'] = 'SUV_&_Van'
dp.loc[(dp['class']=='Adult') | (dp['class']=='Kid') | (dp['class']=='Kickboard'), 'class'] = 'Person'
dp.loc[(dp['class']=='Small_Truck') | (dp['class']=='Medium_Truck') | (dp['class']=='Large_Truck'), 'class'] = 'Truck'
dp.loc[(dp['class']=='Mini_Bus') | (dp['class']=='Bus'), 'class'] = 'Bus'

qt = dp[['class', 'l', 'w', 'h']].groupby('class').quantile(0.99)
dp.loc[(dp['class']=='Special_Vehicle') & (dp['l']<=qt.loc['SUV_&_Van']['l']) & (dp['w']<=qt.loc['SUV_&_Van']['w']) & (dp['h']<=qt.loc['SUV_&_Van']['h']), 'class'] = 'SUV_&_Van'
dp.loc[(dp['class']=='Special_Vehicle') & (dp['l']>qt.loc['SUV_&_Van']['l']) & (dp['w']>qt.loc['SUV_&_Van']['w']) & (dp['h']>qt.loc['SUV_&_Van']['h'])
        & (dp['l']<=qt.loc['Truck']['l']) & (dp['w']<=qt.loc['Truck']['w']) & (dp['h']<=qt.loc['Truck']['h']), 'class'] = 'Truck'

# # yolov5
---

In [80]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final/yolov5_integ'

os.makedirs(f'{dst}/labels', exist_ok=True)
os.makedirs(f'{dst}/images', exist_ok=True)
os.makedirs(f'{dst}/ImageSets', exist_ok=True)
os.makedirs(f'{dst}/test_images', exist_ok=True)

class_num = {'Car': 0,
             'SUV_&_Van': 1,
             'Truck': 2,
             'Bus': 3,
             'Special_Vehicle': 4,
             'Two_Wheeler': 5,
             'Person': 6}

dat_typs = []

scenes = dp['scene'].unique()
for scene in tqdm(scenes):
    dat_typs.append(re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0])

    frames = dp.loc[dp['scene']==scene, 'frame'].unique()
    for frame in frames:

        # make points
        frame_data = dp.loc[(dp['scene']==scene) & (dp['frame']==frame)].copy()
        frame_data['class_num'] = frame_data['class'].apply(lambda x: class_num[x])
        xyxy_ls = frame_data[['min_x', 'min_y', 'max_x', 'max_y']].values
        
        xywhn_ls = []
        for xyxy in xyxy_ls:
            xywhn = xyxy2xywhn(xyxy)
            xywhn_ls.append(xywhn)
        
        frame_data[['xn', 'yn', 'wn', 'hn']] = xywhn_ls
        frame_data[['class_num', 'xn', 'yn', 'wn', 'hn']].to_csv(f'{dst}/labels/{scene}_{frame}.txt', header=False, index=False, sep=' ')

        # make images
        image_src = f'{src}/{scene}/camera/camera_0/{frame}.jpg'
        image_dst = f'{dst}/images/{scene}_{frame}.jpg'
        shutil.copyfile(image_src, image_dst)

# make ImageSets
train_ls = []
val_ls = []
test_ls = []

images = sorted(glob.glob(f'{dst}/images/*.jpg'))
# dat_typs = set([re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0] for scene in scenes])
for dat_typ in sorted(set(dat_typs)):
    scenes_typ = [scene for scene in scenes if dat_typ in scene]
    
    train_val, test = train_test_split(scenes_typ, test_size=0.1, shuffle=False, random_state=44)
    train, val = train_test_split(train_val, test_size=0.2, random_state=44)

    for j in train:
        for image in images:
            if j in image:
                train_ls.append(image)

    for j in val:
        for image in images:
            if j in image:
                val_ls.append(image)

    for j in test:
        for image in images:
            if j in image:
                test_ls.append(image)

with open(f'{dst}/ImageSets/train.txt', 'w') as f:
    f.write('\n'.join(sorted(train_ls)))
    
with open(f'{dst}/ImageSets/val.txt', 'w') as f:
    f.write('\n'.join(sorted(val_ls)))

with open(f'{dst}/ImageSets/test.txt', 'w') as f:
    f.write('\n'.join(sorted(test_ls)))

# make test_images
with open(f'{dst}/ImageSets/test.txt', 'r') as f:
    test_images = [j.replace('\n', '') for j in f.readlines()]

for image in test_images:
    if os.path.exists(image) == True:
        shutil.move(image, f'{dst}/test_images')

100%|██████████| 4886/4886 [1:25:52<00:00,  1.05s/it]  


# # pvrcnn
---

In [84]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final/pvrcnn_integ'

os.makedirs(f'{dst}/labels', exist_ok=True)
os.makedirs(f'{dst}/points', exist_ok=True)
os.makedirs(f'{dst}/ImageSets', exist_ok=True)

scenes = dp['scene'].unique()
for scene in tqdm(scenes):
    # dat_typs.append(re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0])

    # with open(f'{src}/{scene}/calib/camera/camera_0.json', 'r') as f:
    #     calib = json.load(f)

    # for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
    #     if calib['extrinsic'] == calib_typ[typ]['calib']:
    #         mov_zpoint = calib_typ[typ]['mov_zpoint']

    frames = dp.loc[dp['scene']==scene, 'frame'].unique()
    for frame in frames:

        # make labels
        frame_data = dp.loc[(dp['scene']==scene) & (dp['frame']==frame)].copy()
        frame_data[['point_x', 'point_y', 'point_z', 'l', 'w', 'h', 'rot_y', 'class']].to_csv(f'{dst}/labels/{scene}_{frame}.txt', header=False, index=False, sep=' ')

 44%|████▍     | 2163/4886 [23:43<29:52,  1.52it/s] 


KeyboardInterrupt: 

In [ ]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final/pvrcnn_integ'

os.makedirs(f'{dst}/labels', exist_ok=True)
os.makedirs(f'{dst}/points', exist_ok=True)
os.makedirs(f'{dst}/ImageSets', exist_ok=True)

# # Z축 이동을 위해서 calib와 매칭하여 이동범위 지정
# calib_ls = []
# # scenes = []
# calibs = sorted(glob.glob(f'{src}/*/calib/camera/camera_0.json'))
# for calib in calibs:
#     scene = re.findall('[a-zA-Z0-9_]+', calib)[-5]
#     with open(calib, 'r') as f:
#         calib = json.load(f)
#     if calib['extrinsic'] not in calib_ls:
#         calib_ls.append(calib['extrinsic'])
#         # scenes.append(scene)    
# calib_typ = {'typ1': {'calib': calib_ls[0], 'mov_zpoint': 14},
#              'typ2': {'calib': calib_ls[1], 'mov_zpoint': 13},
#              'typ3': {'calib': calib_ls[2], 'mov_zpoint': 0},
#              'typ4': {'calib': calib_ls[3], 'mov_zpoint': -20}}

dat_typs = []

scenes = dp['scene'].unique()
for scene in tqdm(scenes):
    dat_typs.append(re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0])

    # with open(f'{src}/{scene}/calib/camera/camera_0.json', 'r') as f:
    #     calib = json.load(f)

    # for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
    #     if calib['extrinsic'] == calib_typ[typ]['calib']:
    #         mov_zpoint = calib_typ[typ]['mov_zpoint']

    frames = dp.loc[dp['scene']==scene, 'frame'].unique()
    for frame in frames:

        # make labels
        frame_data = dp.loc[(dp['scene']==scene) & (dp['frame']==frame)].copy()
        frame_data[['point_x', 'point_y', 'point_z', 'l', 'w', 'h', 'rot_y', 'class']].to_csv(f'{dst}/labels/{scene}_{frame}.txt', header=False, index=False, sep=' ')

        # make points
        point_src = f'{src}/{scene}/lidar/{frame}.pcd'
        point_dst = f'{dst}/points/{scene}_{frame}.npy'

        pcd = o3d.t.io.read_point_cloud(point_src)
        positions = pcd.point.positions.numpy()
        intensity = pcd.point.intensity.numpy()
        positions[:, 2] += frame_data['mov_point_z'].values[0]

        pcd = np.concatenate((positions, intensity), axis = 1)
        np.save(point_dst, pcd)

# make ImageSets
train_ls = []
val_ls = []
test_ls = []

points = sorted(glob.glob(f'{dst}/points/*.npy'))
for dat_typ in sorted(set(dat_typs)):
    # images_typ = [image for image in images if dat_typ in image]
    scenes_typ = [scene for scene in scenes if dat_typ in scene]
    
    train_val, test = train_test_split(scenes_typ, test_size=0.1, shuffle=False, random_state=44)
    train, val = train_test_split(train_val, test_size=0.2, random_state=44)

    for j in train:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                train_ls.append(point)

    for j in val:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                val_ls.append(point)

    for j in test:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                test_ls.append(point)

with open(f'{dst}/ImageSets/train.txt', 'w') as f:
    f.write('\n'.join(sorted(train_ls)))
    
with open(f'{dst}/ImageSets/val.txt', 'w') as f:
    f.write('\n'.join(sorted(val_ls)))

with open(f'{dst}/ImageSets/test.txt', 'w') as f:
    f.write('\n'.join(sorted(test_ls)))

## # deepfusionmot
---